#### Table Replication  

In [18]:
import bq, config
from google.cloud import bigquery

In [2]:
fp = "./sc/bq_h_01.sql"

In [3]:
query_text = ""
with open(fp, "r") as f:
    query_text = f.read()

In [4]:
def create_table_remix(fp, source, destination):
    with open(fp, "r") as f:
        query_text = f.read()
    query_text = query_text.replace("_source_table", source)
    query_text = query_text.replace("_destination_table", destination)
    return query_text

In [9]:
project = "tpc-benchmarking-9432"
source_dataset = "h_100GB_01" 
destination_dataset = "h_100GB_02"

In [10]:
query_text = create_table_remix(fp=fp,
                                source=source_dataset,
                                destination=destination_dataset)

In [11]:
bq.create_dataset(destination_dataset)

Dataset(DatasetReference('tpc-benchmarking-9432', 'h_100GB_02'))

In [20]:
client = bigquery.Client.from_service_account_json(config.gcp_cred_file)

job_config = bigquery.QueryJobConfig()
#job_config.default_dataset = config.gcp_project.lower() + "." + dataset

query_job = client.query(query_text, job_config=job_config)  # API request
rows = query_job.result()  # Waits for query to finish

NameError: name 'verbose' is not defined

In [21]:
for r in rows:
    print(r.name)

In [22]:
rows